# 🚀 Databricks Project Setup & Pipeline Guide: alldatatech_project

This document provides step-by-step instructions for setting up the environment, ingesting initial data, and configuring the monthly extraction pipeline within **Databricks** using **Unity Catalog**.

---

## 1. ⚙️ Initial Environment Setup & Data Ingestion

The steps below are executed using the **`alldatatech_project Setup and Data Ingestion.ipynb`** notebook.

### 1.1 Pre-Requisite Actions

1.  **Download Files:** Download all necessary project files from the **GitHub** repository to your local machine.
2.  **Import Notebook:** Import the notebook named **`alldatatech_project Setup and Data Ingestion.ipynb`** into your Databricks workspace.

### 1.2 Notebook Execution Steps

Execute the query cells in the imported setup notebook **sequentially** to establish the necessary database objects and prepare for data ingestion.

| Step # | Action | Description |
| :--- | :--- | :--- |
| **1** | Creating a New Spark **Catalog** | Defines the top-level namespace (e.g., `alldatatech_project`) in Unity Catalog. |
| **2** | Creating a New Spark **Schema** (Database) | Creates a schema within the catalog (e.g., `sales_reporting`) to organize tables. |
| **3** | Creating the **`enrollment_status_log`** Delta Table | Defines the target Delta Lake table for the ingested data. |
| **4** | Creating a Unity Catalog **Volume** | Sets up a Unity Catalog **Volume** to manage access to cloud storage for raw files (e.g., CSVs). |
| **5** | **Upload Sample Data** (Manual) | **MANUAL ACTION:** Upload the file **`sample_enrollment_data.csv`** to the following Volume path: **`alldatatech_project.sales_reporting.enrollment_files`**. |
| **6** | PySpark: Ingesting CSV Data | Reads the uploaded CSV file from the Volume and loads it into the **`enrollment_status_log`** Delta table. |
| **7** | Viewing All Data | Validates successful ingestion by displaying the contents of the target table. |
| **8** | Creating a Subdirectory | Creates a new subdirectory within the Volume path (e.g., for archiving processed files). |

---

## 2. 🚀 Configuring the Extraction Pipeline

This section configures the monthly extraction process using a Databricks Job pipeline.

### 2.1 Job Creation via YAML

1.  **Navigate to Jobs:** Go to **Workflows** (or Jobs & Pipelines) in the Databricks workspace sidebar.
2.  **Create Job:** Click **"Create Job"**.
3.  **Edit as YAML:** Switch the job creation interface to the **"Edit as YAML"** view.
4.  **Paste YAML Code:** Copy the entire content of the **`wf_extracts_monthly.yml`** file and paste it into the YAML editor.
5.  **Save:** Save the job configuration. This creates the pipeline named **`wf_extracts_monthly`**.

### 2.2 Notebook Preparation

1.  **Import Extraction Notebook:** Import the notebook named **`Extract.ipynb`** into your Databricks workspace.
2.  **Update Pipeline Path:**
    * Open the newly created job **`wf_extracts_monthly`**.
    * Inspect the notebook task within the job.
    * **Update the Notebook Path** in the job task definition to point to the exact location of the imported **`Extract.ipynb`** file in your workspace.

### 2.3 Running the Pipeline

1.  **Execute Job:** Open the **`wf_extracts_monthly`** job.
2.  Click **"Run now"** to execute the pipeline and initiate the data extraction process.

---

## Key Artifacts Summary

| Artifact | Type | Purpose |
| :--- | :--- | :--- |
| **`alldatatech_project Setup and Data Ingestion.ipynb`** | Notebook | One-time setup of UC Catalog, Schema, Volume, and Delta Table. |
| **`sample_enrollment_data.csv`** | Raw Data File | Initial data source uploaded to the Volume. |
| **`wf_extracts_monthly.yml`** | YAML Definition | Configuration file for the Databricks Job (Pipeline). |
| **`Extract.ipynb`** | Notebook | Contains the primary ETL/extraction logic run by the job. |

In [0]:
spark.sql("CREATE CATALOG alldatatech_project")

In [0]:
spark.sql("CREATE SCHEMA alldatatech_project.sales_reporting")

In [0]:
 %sql
CREATE TABLE alldatatech_project.sales_reporting.enrollment_status_log (
  fes_enrollment_id STRING COMMENT 'The Unique enrollment Id',
  fes_revision_identifier STRING COMMENT 'Revision Identifier changes every time there is an update to the record',
  enrollment_update_ts TIMESTAMP COMMENT 'The date this record was last modified',
  dealer_name STRING COMMENT 'The dealership name',
  pacode STRING COMMENT 'The Part and Accessory Code that uniquely identifies a dealer',
  country STRING COMMENT 'The country determines if the record is from US or Canada',
  dealer_status STRING COMMENT 'The dealer status : Active, Terminated, Enrolled ,etc',
  product_name STRING COMMENT 'The Name of the Product the dealer is enrolled into',
  sku_name STRING COMMENT 'The name of the SKU. Every product is sub categorized based on SKU',
  product_key STRING COMMENT 'The key or code of the Product the dealer is enrolled into',
  sku_key STRING COMMENT 'The key or code of the SKU. Every product is sub categorized based on SKU',
  product_and_sku_key STRING COMMENT 'This combination is unique for a given product and Sku',
  sku_type STRING COMMENT 'The type of product, whether its an Add-on, Tier, etc',
  brand_type STRING COMMENT 'The brand type of the dealer: Dual/Rooftop , Ford, Lincoln, Quicklane',
  brand_name STRING COMMENT 'The Brand Name of the dealer.Can be Ford, Lincoln or QuickLane',
  enrollment_type STRING COMMENT 'The Enrollment Type',
  enrollment_status STRING COMMENT 'The Enrollment Status : Enrolled , Terminated',
  enrollment_secondary_status STRING COMMENT 'The Secondary status of the enrollment, Active, Inactive',
  quantity INT COMMENT 'The number of enrollments bought by the dealer, Mostly 1',
  one_time_fee DECIMAL(10,0) COMMENT 'The one-time fee for the product enrollment',
  monthly_fee DECIMAL(10,0) COMMENT 'The montly fee for the product enrollment. Most of the products have monthly fees',
  annual_fee DECIMAL(10,0) COMMENT 'The annual fee of the product enrollment',
  fields STRING COMMENT 'The additional details with respect to product enrollment',
  enrollment_ts TIMESTAMP COMMENT 'The enrollment date of the dealer for the product',
  activation_ts TIMESTAMP COMMENT 'The activation date of the product',
  termination_ts TIMESTAMP COMMENT 'The date the dealer terminated form the product',
  termination_reason STRING COMMENT 'The reason why the dealer terminated from the productd',
  primary_contact_name STRING COMMENT 'The primary contact name for the product',
  primary_contact_title STRING COMMENT 'The Title of the primary contact name for the product',
  primary_contact_phone STRING COMMENT 'The phone number of the primary contact for the product', -- Added missing COMMENT for phone
  primary_contact_email STRING COMMENT 'The email of the primary contact for the product',
  create_user STRING COMMENT 'The user that created this record',
  create_epoch BIGINT COMMENT 'The create date epoch',
  create_ts STRING COMMENT 'The create date timestamp',
  update_user STRING COMMENT 'The user that updated this record',
  update_epoch BIGINT COMMENT 'The update date epoch ',
  update_ts STRING COMMENT 'The update date timestamp',
  file_date STRING COMMENT 'The date of the file or the record'
)
USING delta
COMMENT 'Log of dealer enrollment statuses for sales reporting.' -- Added a descriptive table comment
TBLPROPERTIES (
  'delta.minReaderVersion' = '1',
  'delta.minWriterVersion' = '2',
  'external.table.purge' = 'true'
  -- Removed extraneous or legacy TBLPROPERTIES like 'STATS_GENERATED' and 'bucketing_version' for a clean new UC table
);

In [0]:
spark.sql("CREATE VOLUME alldatatech_project.sales_reporting.enrollment_files")

### MANUAL ACTION: Upload the file sample_enrollment_data.csv to the following Volume path: alldatatech_project.sales_reporting.enrollment_files

In [0]:
# Read CSV into DataFrame
df = spark.read.format("csv") \
  .option("header", "true") \
  .option("nullValue", "null") \
  .option("inferSchema", "true") \
  .load("/Volumes/alldatatech_project/sales_reporting/enrollment_files/sample_enrollment_data.csv") 
display(df)
# print("Columns in DataFrame:", df.columns)

# Insert data into existing table
df.write \
    .mode("append") \
    .insertInto("alldatatech_project.sales_reporting.enrollment_status_log")

In [0]:
%sql
select * from alldatatech_project.sales_reporting.enrollment_status_log

In [0]:
dbutils.fs.mkdirs("/Volumes/alldatatech_project/sales_reporting/enrollment_files/reporting")